# L03 Kata - Classification

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pickle import *
from fastai.vision import *

### Exploring the data

In [ ]:
path = untar_data(URLs.BIWI_SAMPLE)
path

In [ ]:
fn2ctr = pickle.load(open(path/'centers.pkl', 'rb'))

In [ ]:
fname = '1.jpg'

In [ ]:
img = open_image(path/'images'/fname)
img.show()

In [ ]:
def get_ctr(f):
    return fn2ctr[f]

def get_ip(img,pts): 
    return ImagePoints(FlowField(img.size, pts), scale=True)

In [ ]:
get_ctr(fname)

In [ ]:
ctr = fn2ctr[fname]
img.show(y=get_ip(img, ctr), figsize=(6, 6))

### Loading the data

In [ ]:
np.random.seed(42)
tfms = get_transforms()

data = (PointsItemList.from_folder(path)
        .split_by_rand_pct(0.2)
        .label_from_func(lambda o:fn2ctr[o.name])
        .transform(tfms, tfm_y=True, size=(120,160))
        .databunch()
        .normalize(imagenet_stats))

In [ ]:
data.show_batch(3, figsize=(9,6))

### Training

In [ ]:
learn = cnn_learner(data, models.resnet34, lin_ftrs=[100], ps=0.05)
learn.loss_func = MSELossFlat()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 2e-02

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1')

In [ ]:
learn.show_results()

### Accuracy analysis

### Predict on CPU